In [ ]:
# Import needed libraries and packages
import numpy as np
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

### Question 1

For each point in the complex plane $c = x + iy$, with $-2 < x < 2$, set $z_{0} = 0$ and $-2 < y < 2$ and iterate the equation $z_{i+1} = z_{i}^2 + c$. Note what happens to the $z_{i}$'s: some points will remain bounded in absolute value $|z|^2 = \Re(z)^2 + \Im(z)^2$, while others will run off to infinity. 

Make an image in which your points $c$ that diverge are given one colour and those that stay bounded are given another.

Make a second image where the points are coloured by a colourscale that indicates the iteration number at which the given point diverged.

In [ ]:
# Import the function that performs the Mandelbrot iteration
from mandelbrot import *

In [ ]:
# Initialize list to contain needed points in the complex plane
complex_plane = []

# Fill list with needed points
a = np.linspace(-2, 2, 1000)
b = np.linspace(-2, 2, 1000)

for x in a:
    for y in b:
        c = x + 1j * y
        complex_plane.append(c)

In [ ]:
# Initialize the sets that will contain the points in the complex plane
# that are bounded and diverge respectfully
bounded = []
diverges = []

# Fill the lists with the appropriate points
for c in complex_plane:
    if mandelbrot(c) < 100:  # Check if c diverges
        diverges.append(c)
    else:                 
        bounded.append(c)

# Create lists that contain the real and imaginary parts of all the 
# bounded points
real_b = [num.real for num in bounded]
imag_b = [num.imag for num in bounded]

# Create lists that contain the real and imaginary parts of all the 
# divergent points
real_d = [num.real for num in diverges]
imag_d = [num.imag for num in diverges]

# Plot an image of all the points in the complex plane, colour-coded
# based on whether or not they diverge
plt.figure(figsize=(6,6))
plt.scatter(real_b, imag_b, color = "tab:blue", label = "bounded") 
plt.scatter(real_d, imag_d, color = "tab:red", label = "diverges") 
plt.ylabel('Imaginary') 
plt.xlabel('Real') 
plt.legend(loc='upper right')
plt.savefig("mandelbrot.pdf", format="pdf", bbox_inches="tight") 
plt.show() 

In [ ]:
# Create a 1000 x 1000 array filled with zeroes
iterations = np.zeros((1000, 1000))

# Fill the array with return value of iterate for each point in the 
# complex plane
for i in range(1000):
        for j in range(1000):
            iterations[i, j] = mandelbrot(a[j] + 1j * b[i])

# Plot all the points in colourscale, indicating the iteration number at
# which a point diverged
plt.figure(figsize=(6,6))
plt.imshow(iterations, cmap="winter", extent=(-2, 2, -2, 2))
plt.colorbar(label='Iterations until Divergence')
plt.xlabel('Real')
plt.ylabel('Imaginary')
plt.savefig("mandelbrot_colourscale.pdf", format="pdf", bbox_inches="tight") 
plt.show()

### Question 2

Lorenz was interested in modeling the behavior of Earth's atmosphere. Lorenz applies a Fourier transform to the basic equations, and truncates the number of Fourier modes, keeping only three, with amplitudes denoted by $W\equiv(X, Y, Z)$.

The equations (Lorenz' equations 25, 26, and 27) are

\begin{eqnarray}
\dot X &=& -\sigma(X-Y)\\
\dot Y &=& rX -Y - XZ\\
\dot Z &=& -bZ + XY
\end{eqnarray}

The three dimensionless parameters are $\sigma$, the Prandtl number (the ratio of the kinematic viscosity to the thermal diffusivity), the Rayleigh number $r$ (which depends on the vertical temperature difference between the top and bottom of the atmosphere), and b, which is a dimensionless length scale.

Note that there are non-linear terms in the second and third equations;
these terms result in very complex dynamics.

Your task is to 
1. code up the equations, using a function definition, with a proper docstrings (inside triple quotes)
2. use an ode solver of your choice, i.e., solve\_ivp, or ode, to integrate the equations for t=60 (in dimensionless time units). Use Lorenz' initial conditions $W_0=[0., 1., 0.]$ and his parameter values [$\sigma, r, b$] = [10., 28, 8./3.].
3. reproduce Lorenz' Figure 1. Label both axes! Note that Lorenz uses $N=t/\Delta t$ to label his plots (here $\Delta t=0.01$).
4. reproduce Lorenz' Figure 2. You will likely have to ask for output at very closely spaced time intervals, e.g., if you use solve\_ivp, you will need something like t = np.linspace(14, 19, 1000) followed by W = sol.sol(t). Again, label both axes.
5. find the solution using the same values of $(\sigma, r, b)$, but this time with initial conditions very slightly different than $W_0$, say $W'_0 = W_0+[0., 1.e-8, 0] = [0., 1.00000001, 0.]$; note that adding the two lists (as indicated here) will not work, so you should google to find out how to add two lists element by element. Calculate the distance between $W'$ and $W$ as a function of time, and plot the result on a semilog plot (linear time, log distance). 



In [ ]:
# Import the function that defines the needed Lorenz equations
from lorenz_equations import *

In [ ]:
# PART 2

# Lorenz parameters
sig, r, b = 10.0, 28.0, 8.0/3.0

# Time span
t_span = (0, 60)

# Initial conditions
w0 = [0.0, 1.0, 0.0]

# Integrate the equations
sol = solve_ivp(lorenz_equations, t_span, w0, args=(sig, r, b), t_eval=np.linspace(0, 60, 1000))

# Access the solution at t = 60
t_60 = sol.t[-1]
w_t_60 = sol.y[:, -1]

print("The solution to the system at t = 60:", w_t_60)

In [ ]:
# PART 3

# Time span
t_span = (0, 30)

# Time step
dt = 0.01

# Integrate the equations
sol = solve_ivp(lorenz_equations, t_span, w0, args=(sig, r, b), t_eval=np.linspace(0, 30, 1000), max_step=dt)

# Number of iterations
n = sol.t/dt

# Plot y as a function of time
fig = plt.figure(figsize = (10,8))

# Plot y as a function of time from 0 to 1000 iterations
plt.subplot(3,1,1) 
plt.xlabel('Iterations')
plt.plot(n, sol.y[1])
plt.xlim(0,1000)
plt.ylabel('y')
plt.grid(True)

# Plot y as a function of time from 1000 to 2000 iterations
plt.subplot(3,1,2) 
plt.plot(n, sol.y[1])
plt.xlabel('Iterations')
plt.xlim(1000,2000)
plt.ylabel('y')
plt.grid(True)

# Plot y as a function of time from 2000 to 3000 iterations
plt.subplot(3,1,3) 
plt.plot(n, sol.y[1])
plt.xlabel('Iterations')
plt.xlim(2000,3000)
plt.ylabel('y')
plt.grid(True)

plt.tight_layout()
plt.savefig("lorenz_fig_1.pdf", format="pdf", bbox_inches="tight") 
plt.show()

In [ ]:
# PART 4

# Time span 
t_span = (0, 60)

# Integrate the equations using solve_ivp
sol = solve_ivp(lorenz_equations, t_span, w0, args=(sig, r, b), t_eval=np.linspace(0, 60, 10000))

# Extract the segment of the trajectory from iteration 1400 to 1900
t_seg = (sol.t >= 14) & (sol.t <= 19)
w_seg = sol.y[:, t_seg]
                                
# Plot the projections onto xy and yz planes
plt.figure(figsize=(10, 5))

# Plot xy projection
plt.subplot(1, 2, 1)
plt.plot(w_seg[1], w_seg[0], color='tab:blue')
plt.xlabel('y')
plt.ylabel('x')
plt.title('Projection onto XY plane')
plt.grid(True)

# Plot yz projection
plt.subplot(1, 2, 2)
plt.plot(w_seg[1], w_seg[2], color='tab:red')
plt.xlabel('y')
plt.ylabel('z')
plt.title('Projection onto YZ plane')
plt.grid(True)

plt.tight_layout()
plt.savefig("lorenz_fig_2.pdf", format="pdf", bbox_inches="tight") 
plt.show()

In [ ]:
# PART 5

# Initial conditions (w0 was defined in PART 2)
w0_prime = np.array(w0) + np.array([0., 1e-8, 0.])  

# Time span
t_span = (0, 60)

# Integrate the equations with initial conditions w0
sol_w = solve_ivp(lorenz_equations, t_span, w0, args=(sig, r, b), t_eval=np.linspace(0, 60, 10000))

# Integrate the equations with initial conditions w0_prime
sol_w_prime = solve_ivp(lorenz_equations, t_span, w0_prime, args=(sig, r, b), t_eval=np.linspace(0, 60, 10000))

# Calculate the distance between W' and W at each time step
distance = np.linalg.norm(sol_w_prime.y - sol_w.y, axis=0)

# Plot the distance as a function of time on a semilog plot
plt.semilogy(sol_w_prime.t, distance)
plt.xlabel('Time')
plt.ylabel('Distance')
plt.title('Distance between W\' and W as a function of time')
plt.grid(True)
plt.savefig("distance.pdf", format="pdf", bbox_inches="tight") 
plt.show()